In [1]:
from sharadar.pipeline.engine import load_sharadar_bundle
from sharadar.loaders.ingest_macro import ingest

In [2]:
bundle = load_sharadar_bundle()

In [3]:
start = bundle.equity_daily_bar_reader.first_trading_day
end = bundle.equity_daily_bar_reader.last_available_dt
start, end

(Timestamp('1997-12-31 00:00:00+0000', tz='UTC'),
 Timestamp('2021-04-16 00:00:00+0000', tz='UTC'))

In [4]:
ingest(start, end)

101372